# LangGraph를 사용한 고객 지원 챗봇을 구축

- 웹 검색을 통해 일반적인 질문에 답변 
- 대화 상태를 유지하여 연속적인 대화  
- 복잡한 질문을 사람이 검토하도록 라우팅  
- 사용자 지정 상태(Custom State)를 활용하여 챗봇의 동작 제어  
- 대화 흐름을 되돌리고(Rewind), 다른 대화 경로 탐색 

In [ ]:
# LangSmith 추적 설정 활성화

먼저 모델을 직접 사용해 봅니다. `ChatModel`은 LangChain의 **"Runnable"** 인스턴스이며, 이는 표준화된 인터페이스를 통해 상호작용할 수 있음을 의미합니다.  

모델을 간단하게 호출하려면 `.invoke` 메서드에 **메시지 목록**을 전달하면 됩니다.

모델 자체는 **상태(state)** 라는 개념을 가지고 있지 않습니다. 예를 들어, 후속 질문을 하면:

<br>
이제 좋은 응답을 받는 것을 확인할 수 있습니다!  

이것이 챗봇이 **대화형 상호작용**을 할 수 있는 기본 아이디어입니다.  

## 챗봇에 메모리 기능 추가¶

현재 챗봇은 **사용자 질문에 도구를 활용해 답변할 수 있지만, 이전 대화의 맥락을 기억하지 못합니다.**  
이 때문에 **일관된 멀티턴(Multi-turn) 대화를 진행하는 데 한계가 있습니다.**  

LangGraph는 **"지속적 체크포인트(Persistent Checkpointing)"** 기능을 통해 이 문제를 해결합니다.  

그래프를 컴파일할 때 checkpointing을 활성화하고 그래프를 호출할 때 `thread_id`를 제공하면, LangGraph가 자동으로 상태(state)를 저장하고, 다음 실행 시 이전 상태를 복원합니다.  

즉, **동일한 `thread_id`** 를 사용하여 그래프를 호출하면, 이전 대화 상태를 불러와서 이어서 대화할 수 있습니다!

우리는 현재 **메모리를 활용하는(in-memory) 체크포인터**를 사용하고 있습니다.  

이 방식은 튜토리얼 환경에서는 편리하지만, 데이터가 메모리에만 저장되므로 영구적이지 않습니다. 실제 프로덕션 환경에서는 `SqliteSaver` 또는 `PostgresSaver`를 사용하여 데이터베이스(DB)와 연결하는 것이 일반적입니다.

---

In [ ]:
# Define a new graph
# Define the function that calls the model
def call_model(state: MessagesState):
# Define the (single) node in the graph
# Add memory

이제 챗봇과 상호작용할 수 있습니다. 먼저, 이 대화를 식별할 수 있는 **`thread`(스레드)** 를 선택합니다.

In [ ]:
# config는 stream() 또는 invoke()의 두 번째 인자

이제 후속 질문(follow-up question)을 해봅시다. 챗봇이 사용자의 이름을 기억하는지 확인해 봅니다.

### 프롬프트 템플릿(Prompt Templates)을 사용하여 LLM 호출 최적화

**프롬프트 템플릿** 은 **원시 사용자 입력(raw user input)** 을 LLM이 처리할 수 있는 형식으로 변환하는 데 도움을 줍니다.   

1. 먼저, **시스템 메시지(system message)** 를 추가하여 **사용자 정의 지침(custom instructions)** 을 포함시킵니다. (여전히 메시지를 입력으로 사용)  
2. 다음으로, 메시지 외에 **더 많은 입력 정보** 를 추가합니다.  

#### **시스템 메시지(System Message) 추가하기**

시스템 메시지를 추가하기 위해 **`ChatPromptTemplate`** 을 생성합니다. 여기서는 메시지 전달을 위해 **`MessagesPlaceholder`** 를 사용하겠습니다.  

이렇게 하면 LLM에 전달되는 입력이 더 구조화되고, 챗봇의 동작을 더 정교하게 제어할 수 있습니다.  

```
┌─────────────────────────────┐
│       LangGraph State       │
│  MessagesState = {"messages": [...] } │
└─────────────────────────────┘
             │
             ▼
 ┌────────────────────────────────────┐
 │ ChatPromptTemplate.from_messages  │
 │ ┌──────────────────────────────┐ │
 │ │ "system": "친구처럼 말해줘"     │ │
 │ │ MessagesPlaceholder("messages") │◄─── state["messages"]
 │ └──────────────────────────────┘ │
 └────────────────────────────────────┘
             │
             ▼
    llm.invoke(prompt) 수행
             │
             ▼
 ┌──────────────────────────────┐
 │   LLM 응답: AIMessage(...)   │
 └──────────────────────────────┘
             │
             ▼
 ┌──────────────────────────────┐
 │ return {"messages": response}│ ──► 다음 Graph Step 으로 상태 전달
 └──────────────────────────────┘
```

In [ ]:
# LLM이 사용자 입력을 더 잘 처리할 수 있도록 프롬프트 템플릿을 설정합니다.
        # LLM의 동작 방식을 정의하는 지침
        # Messages Placeholder - 이전 대화 메시지들을 전달합

이제 이 템플릿을 통합하여 애플리케이션을 업데이트할 수 있습니다.

In [ ]:
# 상태 스키마(State)를 정의하여 그래프 생성기 초기화
# 모델 호출 함수 정의
def chatbot(state: MessagesState):
    # 현재 상태(state)에 포함된 메시지를 기반으로 프롬프트 생성
    # 생성된 프롬프트를 LLM에 전달하여 응답을 생성
    # LLM 응답을 딕셔너리 형태로 반환 (다음 상태로 전달됨)
# chatbot 노드를 그래프에 추가
# MemorySaver를 사용하여 대화 상태를 저장할 메모리 객체 생성
# 그래프의 시작 지점(START)에서 chatbot 노드로 연결
# 모든 구성 요소를 종합하여 그래프를 컴파일 (메모리를 체크포인터로 사용)
# 컴파일된 그래프 객체를 확인

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 메시지 목록에 사용자 메시지 추가
# 애플리케이션 호출
# 메시지 상태(State), 설정(config) 전달

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 메시지 목록에 사용자 메시지 추가
# 애플리케이션 호출
# 메시지 상태(State), 설정(config) 전달

### 무한 loop 로 Chatbot 구현

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 그래프를 메모리 체크포인트와 함께 컴파일합니다.

------------
이제는 chatbot이 외부 정보를 접근할 수도 있고 대화 내용을 기억할 수도 있습니다.

### 상태 (State)

- node 와 node 간에 정보를 전달할 때 State 객체데 담아 전달
- TypedDict : 파이썬 dict 에 타입 힌팅을 추가한 개념
- 모든 값을 다 채우지 않아도 된다.
- 새로운 node에서 값을 overwrite 방식으로 채운다.
- Reducer(add_messages 혹은 operator.add): 자동으로 list에 메시지를 추가해 주는 기능

In [ ]:
class GraphState(TypedDict):
# Reducer 사용 예시

In [ ]:
# GraphState 사용 예시
def generate_answer(state: GraphState) -> dict:
    # prompt 구성
    # answer 필드에는 텍스트만 저장
    # messages에는 AIMessage 형태로 저장

In [ ]:
# 실행
# 결과 출력

### 대화 기록 관리
챗봇을 구축할 때 이해해야 할 중요한 개념 중 하나는 대화 기록을 관리하는 방법입니다. 관리하지 않으면 메시지 목록이 무한정 늘어나 LLM 컨텍스트 창을 초과할 수 있습니다. 따라서 전달하는 메시지의 크기를 제한하는 단계를 추가하는 것이 중요합니다.

중요한 점은 프롬프트 템플릿을 사용하기 전, 그리고 메시지 기록에서 이전 메시지를 로드한 후에 이 작업을 수행해야 한다는 것입니다.

메시지 키를 적절히 수정하는 간단한 단계를 프롬프트 앞에 추가한 다음, 해당 새 체인을 Message History class로 래핑하면 됩니다.

LangChain에는 메시지 목록을 관리하기 위한 몇 가지 기본 제공 도우미가 있습니다. 이 경우 trim_messages 도우미를 사용하여 모델로 전송하는 메시지 수를 줄여 보겠습니다. 트리머를 사용하면 보관할 토큰 수와 시스템 메시지를 항상 보관할지 여부, 부분 메시지 허용 여부 등의 매개변수를 지정할 수 있습니다.

In [ ]:
# 메시지 트리머(trimmer) 설정
# 테스트용 메시지 목록: 대화 흐름 예시
# 트리머 호출: 메시지 리스트를 주면 max_tokens 이하로 줄여줍니다
# 결과를 출력하여 어떤 메시지가 유지되었는지 확인

--------------
체인에서 사용하려면 메시지 입력을 프롬프트에 전달하기 전에 트리머를 실행하기만 하면 됩니다.

MessagesState는 챗봇을 위해 대화 기록만을 다루는 간단한 스키마이므로 추가 필드가 필요하면 State(TypedDict) 로 상태 스키마를 정의해야합니다.  
```
class MessagesState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

```

In [ ]:
# --- 상태 스키마 정의 ---
# LangGraph에서 상태(State)는 대화 흐름의 데이터를 저장하는 구조입니다.
# 여기서는 messages (이전 메시지들)와 language (언어 정보)를 포함하는 상태를 정의합니다.
class State(TypedDict):
    # messages: BaseMessage의 시퀀스로 구성된 메시지 리스트.
    # operator.add를 통해 이전 상태에 새로운 메시지를 append 방식으로 병합합니다.
# State 스키마를 기반으로 LangGraph 워크플로우 생성
# 상태를 입력받아 메시지를 트리밍하고, 프롬프트를 구성한 뒤, LLM 호출 결과를 반환하는 함수입니다.
def call_model(state: State):
    # 현재 메시지 리스트를 토큰 수 기준으로 잘라냅니다 (예: max_tokens=65 등)
    # 프롬프트 템플릿에 잘린 메시지와 언어 정보를 넣어 프롬프트 생성
    # 모델 호출 → 결과 메시지 반환
    # LangGraph에서는 다음 상태로 넘어갈 때 딕셔너리 형태로 반환합니다.
# --- 워크플로우 구성 ---
# 대화 상태를 저장하기 위한 메모리 객체를 사용합니다 (기본 제공 MemorySaver).
# 모든 노드와 엣지를 기반으로 LangGraph 앱을 생성합니다.

이제 모델에게 이름을 물어보면 채팅 기록의 해당 부분을 잘라냈기 때문에 모델이 이름을 알 수 없습니다.